# self-Attentionの理解のためのノート

#### P268~P269の式やコードでなぜ特徴量が得られるのだろうと不思議に思う。P268のコードを具体的な値で実行した場合、もともとの画素のどの値がどう計算されたかがわからない。このため、シンボルベースで計算することで計算式からの理解を行いたいと思う。
#### 以下、具体的なコードとなるが、理解は難しかった。具体的な値の動きと、今回の結果を見比べることにより、理解が深まるのかもしれない

## numpy行列の定義。シンボルで扱えるようにする工夫。

In [1]:
from sympy import symbols
import numpy as np

#以下、Chat GPTが生成してくれたコードを少し手直し。

#結果の見やすさのための設定。
np.set_printoptions(linewidth=1000)
#np.set_printoptions(formatter={'all': lambda x: f'{x}, '}) #カンマ
np.set_printoptions(formatter={'all': lambda x: f'{x},     '}) #@@

# 画像のチャネル、幅、高さ
#簡単だけど、全貌がわかりにくいかも
C = 2
W = 2
H = 2

#複雑だが、様子がわかりやすい
#C = 3
#W = 3
#H = 3


# 配列の宣言を行う
image_array = np.empty((C, W, H), dtype=object)

# 配列に値を設定
for c in range(C):
    for w in range(W):
        for h in range(H):
            pixel_value = symbols(f'x_{c}_{w}_{h}')
            image_array[c, w, h] = pixel_value

# 生成した配列を表示
print("Image Array:")
print(image_array)

Image Array:
[[[x_0_0_0,      x_0_0_1,     ]
  [x_0_1_0,      x_0_1_1,     ]]

 [[x_1_0_0,      x_1_0_1,     ]
  [x_1_1_0,      x_1_1_1,     ]]]


## P268をnumpyを使って再現する旅。PyTorchはテンソルの要素が数値のみを許可している。このため、P268をPyTorchではなく、numpyを使って再現する必要がある。
#### ここでもPyTorch→numpyの翻訳にChatGPTは大活躍。。。


In [2]:
print("Xを用意")
X = image_array
print(X.shape)
print(X)
print("Xにバッチ次元を追加する")
X = np.expand_dims(X, axis=0)
print(X.shape)
print(X)
print("XのサイズをB,C,W,H→B,C,Nに変形する")
print( (X.shape[0],X.shape[1], X.shape[2] * X.shape[3]))
X = np.reshape(X, (X.shape[0],X.shape[1], X.shape[2] * X.shape[3]))
print(X.shape)
print(X)
print("掛け算")
print("掛け算：前準備としてX_Tを作る")
X_T = np.transpose(X, (0,2,1))
print(X_T.shape)
print(X_T)
print("掛け算：掛け算の本処理。bmmはmatmulと同じ")
S = np.matmul(X_T, X)
print(S.shape)
print(S)
print("↑について。行は各画素位置(この場合は(0,0),(0,1),(1,0),(1,1)の4パターン)毎に並ぶ。行の中身はその画素位置の各チャネルと、同じチャネルの違う画素位置との掛け算の和が並ぶ")
print("つまり、4*4で16個の要素があり、これが特徴量ということになる。")
print("次に、softmaxを求めるが、PyTorchと違い、numpyには組み込みのsoftmaxが無いため自作する必要がある。以下、Generated By Chat GPT")

def softmax(x, axis=-1):
    e_x = np.exp(x - x.max(axis=axis, keepdims=True))
    return e_x / e_x.sum(axis=axis, keepdims=True)

print("規格化：実際に、softmaxを実行するとエラーになる。umr_maximumという関数でエラーになっているため、これは、おそらく要素が文字列なので、計算不可だからだろう")
print("という訳で、規格化は実際にせずに、規格化されたものとして理解をすすめる")
#m = softmax(S, axis=2)

attention_map_T = S

print("アテンションマップの表示（対称行列なので、転置してもそのままの様子）")
attention_map = np.transpose(attention_map_T, (0,2,1))
print(attention_map.shape)
print(attention_map)

print("self-Attention Mapを計算する")
o = np.matmul(X, np.transpose(attention_map, (0,2,1)))
print(o.shape)
print(o)

Xを用意
(2, 2, 2)
[[[x_0_0_0,      x_0_0_1,     ]
  [x_0_1_0,      x_0_1_1,     ]]

 [[x_1_0_0,      x_1_0_1,     ]
  [x_1_1_0,      x_1_1_1,     ]]]
Xにバッチ次元を追加する
(1, 2, 2, 2)
[[[[x_0_0_0,      x_0_0_1,     ]
   [x_0_1_0,      x_0_1_1,     ]]

  [[x_1_0_0,      x_1_0_1,     ]
   [x_1_1_0,      x_1_1_1,     ]]]]
XのサイズをB,C,W,H→B,C,Nに変形する
(1, 2, 4)
(1, 2, 4)
[[[x_0_0_0,      x_0_0_1,      x_0_1_0,      x_0_1_1,     ]
  [x_1_0_0,      x_1_0_1,      x_1_1_0,      x_1_1_1,     ]]]
掛け算
掛け算：前準備としてX_Tを作る
(1, 4, 2)
[[[x_0_0_0,      x_1_0_0,     ]
  [x_0_0_1,      x_1_0_1,     ]
  [x_0_1_0,      x_1_1_0,     ]
  [x_0_1_1,      x_1_1_1,     ]]]
掛け算：掛け算の本処理。bmmはmatmulと同じ
(1, 4, 4)
[[[x_0_0_0**2 + x_1_0_0**2,      x_0_0_0*x_0_0_1 + x_1_0_0*x_1_0_1,      x_0_0_0*x_0_1_0 + x_1_0_0*x_1_1_0,      x_0_0_0*x_0_1_1 + x_1_0_0*x_1_1_1,     ]
  [x_0_0_0*x_0_0_1 + x_1_0_0*x_1_0_1,      x_0_0_1**2 + x_1_0_1**2,      x_0_0_1*x_0_1_0 + x_1_0_1*x_1_1_0,      x_0_0_1*x_0_1_1 + x_1_0_1*x_1_1_1,     ]
  [x_0_0_0*x_0_1_0

### ↑の結果は、第1次元がバッチ、第2次元がチャネル、第3次元が画素群である(X,Y要素がノベタで並んでいる）。第2次元を見ると、各チャネル毎に確かに、並んでいそうな事もわかる。そして、例えば具体的に[0][P][:]のPの並び(行の並び)を見ると、各チャネルの各画素について、大域的な数値が考慮されているようにも見える。

### 最初の[0][0]を抜き出すと　x_0_0_0*(x_0_0_0**2 + x_1_0_0**2) + x_0_0_1*(x_0_0_0*x_0_0_1 + x_1_0_0*x_1_0_1) + x_0_1_0*(x_0_0_0*x_0_1_0 + x_1_0_0*x_1_1_0) + x_0_1_1*(x_0_0_0*x_0_1_1 + x_1_0_0*x_1_1_1),      である。これは、チャネル0の(0,0)に関するものである。なぜならというと、試しに、"_0_0"でこのノートを文字列検索すると、(0,0)に関するヒット数がこの行で一番多く、(0,0)に関して特徴量を計算しているものと考えられるためである。後の行も同様である。

### この時点で抽象的な理解しかできない。この多項式を見て、ああ、そうだ、特徴を捉えるのに必要な計算式だ。とは到底、理解が追いつかない。実際の値の動きと合わせて見ていくしかないだろう。これを思いついた研究者は偉大としか思えない・・・